In [4]:
import json 

def getmeasurementTimestamp(item):
    return int(item['measurementTimestamp'])

def get_x_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][0]

def get_y_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][1]


for i in [0]:
    data = []
    with open("2015-07-07.json") as f:
        data = f.readlines()

    json_lines = []
    mac_adresses = []
    
    for line in data:
        jsline = json.loads(line)
        json_lines.append(jsline)
        mac_adresses.append(jsline["value"]["sourceMac"])

    

average x error: 21011.453137221877
average y error: 21609.61068440125
h
760


In [8]:
#json_lines.sort(key = getmeasurementTimestamp)
for i in [0]:
    average_x_error = 0
    average_y_error = 0
    count = 0
    good_lines = []
    number_of_people = len(set(mac_adresses))
    mc_good = []
    for jsline in json_lines:
        average_x_error += get_x_error(jsline)
        average_y_error += get_y_error(jsline)
        if get_x_error(jsline) <= 2 and get_y_error(jsline) <= 2 :             
            good_lines.append(jsline)
            mc_good.append(jsline["value"]["sourceMac"])

    number_of_lines = len(json_lines)
    number_of_people_good = len(set(mc_good))
    average_x_error /= number_of_lines
    average_y_error /= number_of_lines
        
    print ('average x error: ' + str(average_x_error))
    print ('average y error: ' + str(average_y_error))
    print('h')
    print('number of people:' + str(number_of_people))     
    print('number of people with small error:' + str(number_of_people_good))     

average x error: 21011.453137221877
average y error: 21609.61068440125
h
number of people:760
number of people with small error:285


In [1]:
285/765.0

0.37254901960784315